# Выделение тем - Камиль Талипов, 294

### LDA на Википедии

In [1]:
import numpy as np
import gensim

Сконвертируем дамп википедии

In [2]:
%%bash
python -m gensim.scripts.make_wiki frwiki-20151202-pages-articles.xml.bz2 wiki_en_output

Обучим lda

In [3]:
id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_output_wordids.txt')
mm = gensim.corpora.MmCorpus('wiki_en_output_tfidf.mm')
model = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word,
                                        num_topics=100, update_every=1,
                                        chunksize=10000, passes=1)
model.save('wiki_lda.pkl')

Посмотрим полученные темы

In [4]:
model.show_topic(99)

[(u'mesa', 0.091435903139672786),
 (u'wta', 0.048343483763978227),
 (u'inde', 0.03505429249916206),
 (u'indien', 0.021937466834919761),
 (u'ha\xefti', 0.016469230400395156),
 (u'litom\u011b\u0159ice', 0.015340198468797712),
 (u'sri', 0.013221911233841754),
 (u'indienne', 0.012911694302291244),
 (u'philippin', 0.011854186555764036),
 (u'lanka', 0.011731928916426626)]

In [5]:
model.show_topic(15)

[(u'orchestre', 0.020230811416973832),
 (u'compositeur', 0.019726689014113279),
 (u'musique', 0.019008675927580914),
 (u'op\xe9ra', 0.01265715301826735),
 (u'piano', 0.012491334974749227),
 (u'conservatoire', 0.0072216815558275842),
 (u'op', 0.0071809523444280538),
 (u'classique', 0.0070684881445231187),
 (u'violon', 0.0067371126627431919),
 (u'pianiste', 0.0064060435131954342)]

In [6]:
model.show_topic(60)

[(u'estonie', 0.030783720653652084),
 (u'sauriens', 0.026252205772809605),
 (u'estonien', 0.020475170084009845),
 (u'tallinn', 0.018723750611277894),
 (u'squamata', 0.016204067251731645),
 (u'serpents', 0.014353648258553224),
 (u'serpent', 0.01399461533409687),
 (u'avex', 0.012898223750635131),
 (u'liolaemidae', 0.011953788568685054),
 (u'sevens', 0.011639298376036566)]

Видно, что первые представители класса очень хорошо связаны логически, а в классе 15 - все совсем хорошо. Учитывая, что мы разбивали всего на 100 классов - это хороший результат.

Обучим модель на 1500 классов

In [2]:
model = gensim.models.ldamodel.LdaModel(
          corpus=mm,
          id2word=id2word,
          num_topics=1500,
          update_every=1,
          chunksize=10000,
          passes=1)
model.save('wiki_lda_big.pkl')

Посмотрим отличия в  моделях

In [3]:
def print_doc_stat(model):
    topics = [model[doc] for doc in mm]
    lens = np.array([len(t) for t in topics])
    print np.mean(lens)
    print np.mean(lens <= 10)

print '1500 model:'
print_doc_stat(model)

print '100 model:'
print_doc_stat(gensim.models.ldamodel.LdaModel.load('wiki_lda.pkl'))

1500 model:
12.922952169
0.304787707929
100 model:
7.18414104802
0.88266542455


Таким образом при выделении 1500 тем в среднее количество тем на документ чуть меньше 13, а 30% документов имеют меньше 10 тем. При выделении 100 тем на документ приходится чуть больше 7 тем и более 88% документов имеют меньше 10 тем. Т

Посмотрим случайные 20 тем

In [4]:
import random

for i in range(20):
    topic_id = random.choice(range(1500))
    print 'Topic #' + str(topic_id)
    print model.show_topic(topic_id)

Topic #1432
[(u'dipl\xf4m\xe9e', 0.038199690496365228), (u'pauline', 0.03056333005274333), (u'monique', 0.023516540523947603), (u'laur\xe9ate', 0.023136796921884063), (u'audrey', 0.019992752491187354), (u'sophie', 0.018247619106246241), (u'c\xe9line', 0.017478963163831397), (u'ir\xe8ne', 0.016263921955023651), (u'fabienne', 0.015163514378865465), (u'brook', 0.013876675678359877)]
Topic #0
[(u'picasso', 0.036438123655694619), (u'gogh', 0.031330938215407923), (u'pablo', 0.030045874362592945), (u'fernand', 0.026912442188809001), (u'oskar', 0.022123907301451889), (u'abstrait', 0.021686066603253142), (u'arte', 0.021398123202320703), (u'matisse', 0.019281029325663144), (u'andy', 0.018806184606631502), (u'grange', 0.018699335903965032)]
Topic #779
[(u'catholic', 0.17632260255558355), (u'thorpe', 0.086256349791197201), (u'viiie', 0.071419780819725762), (u'somerset', 0.063873671529029535), (u'encyclopedia', 0.059129599760745701), (u'aveugles', 0.030707946211204963), (u'intendant', 0.02997786698

Видно, что присутсвуют как хорошо интерпретируемые темы, так и сложно. По сравнению с моделью на 100 тем - все сильно лучше :)

### Word2Vec на Google News

Решил посмотреть какие класстеры присутсвуют в уже обученных векторах их туториала Word2Vec. Они обучались на Google News.

In [7]:
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import MiniBatchKMeans

In [8]:
model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
word_vectors = model.syn0
n_clusters = 1000

kmeans = MiniBatchKMeans(init='k-means++', n_clusters=n_clusters, n_init=3)
idx = kmeans.fit_predict(word_vectors)
word_idx_map = dict(zip(model.index2word, idx))

clusters = []
for cluster_id in xrange(n_clusters):
    words = []
    for i in xrange(len(word_idx_map.values())):
        if (word_idx_map.values()[i] == cluster_id):
            words.append(word_idx_map.keys()[i])
    clusters.append(words)

In [8]:
print clusters[112][:10]

[u'Janessa', u'Crofton_Md._admitted', u'Celia', u'Alizabeth', u'Cari', u'Cara', u'Tinna', u'Hazel', u'Jina', u'Akia']


In [9]:
print clusters[91][:10]

['Ashish_Kapur_CEO', 'Mysore_Cement', 'Sujana_Metal', 'Breweries_Holdings', 'Adani_Exports', 'Grasim_Industries_Ltd', 'Jaypee_Capital', 'IFB_Industries', 'Ashok_Leyland_Tata_Motors', 'saria']


In [10]:
print clusters[784][:10]

['AGNs', 'neutron_stars', 'blazars', 'active_galactic_nuclei', 'X_ray_emission', 'neutron_star', 'quasars', 'supermassive_black_hole', 'galaxies', 'supermassive_black_holes']


Я привел пример хорошо интерпретируеммых классов, но некоторые из полученных классов лично для меня сложно интерпретируемы.